In [26]:
import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from pyspark.sql import SparkSession

import pyspark.sql as pysql

In [ ]:
from pyspark import SparkConf

In [ ]:
conf = SparkConf().setAppName("Tarea3")\
    .setMaster("local[8]") \
    .set("spark.executor.cores","2")
    
spark : SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()

In [27]:
spark : SparkSession= SparkSession.builder.appName("Tarea 3").getOrCreate()

In [28]:
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [29]:
len(columnas)

28

In [30]:
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\data.zip')

path_guardar = pathlib.Path.joinpath(direccion.parent.parent, 'Datos')

archivos_datos = [
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2022.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2021.csv').as_posix()#,
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2020.csv').as_posix(),
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2019.csv').as_posix(),
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2018.csv').as_posix()
]

archivos_datos_en_zip  = [
    #'Combined_Flights_2022.csv',
    'Combined_Flights_2021.csv'#,
    #'Combined_Flights_2020.csv',
    #'Combined_Flights_2019.csv',
    #'Combined_Flights_2018.csv'
]

In [65]:
df = spark.read.csv(archivos_datos[0], header = True)
df = df.select(*columnas)

In [66]:
df = df.withColumn("CRSDepTime", df["CRSDepTime"].cast(pysql.types.FloatType()))
df = df.withColumn("DepTime", df["DepTime"].cast(pysql.types.FloatType()))
df = df.withColumn("DepDelay", df["DepDelay"].cast(pysql.types.FloatType()))
df = df.withColumn("ArrTime", df["ArrTime"].cast(pysql.types.FloatType()))
df = df.withColumn("ArrDelay", df["ArrDelay"].cast(pysql.types.FloatType()))
df = df.withColumn("ActualElapsedTime", df["ActualElapsedTime"].cast(pysql.types.FloatType()))
df = df.withColumn("CRSElapsedTime", df["CRSElapsedTime"].cast(pysql.types.FloatType()))
df = df.withColumn("Distance", df["Distance"].cast(pysql.types.FloatType()))
df = df.withColumn("Year", df["Year"].cast(pysql.types.IntegerType()))
df = df.withColumn("Month", df["Month"].cast(pysql.types.IntegerType()))
df = df.withColumn("DayofMonth", df["DayofMonth"].cast(pysql.types.IntegerType()))
df = df.withColumn("DayOfWeek", df["DayOfWeek"].cast(pysql.types.IntegerType()))

In [67]:
df = df.withColumn("Cancelled", pysql.functions.when(df["Cancelled"] == "False",0).otherwise(1))
df = df.withColumn("Diverted", pysql.functions.when(df["Diverted"] == "False",0).otherwise(1))

In [68]:
df.show(truncate=False)

+---------------------+----------+-------+--------+----------+-------+--------+-----------------+--------------+--------+----+-----+----------+---------+-----------+-------------------------------+------+---------------+------------------+---------------------+---------------+----+-------------+----------------+----------------------------------+-------------+---------+--------+
|Airline              |CRSDepTime|DepTime|DepDelay|CRSArrTime|ArrTime|ArrDelay|ActualElapsedTime|CRSElapsedTime|Distance|Year|Month|DayofMonth|DayOfWeek|Tail_Number|Flight_Number_Operating_Airline|Origin|OriginAirportID|OriginAirportSeqID|OriginCityName       |OriginStateName|Dest|DestAirportID|DestAirportSeqID|DestCityName                      |DestStateName|Cancelled|Diverted|
+---------------------+----------+-------+--------+----------+-------+--------+-----------------+--------------+--------+----+-----+----------+---------+-----------+-------------------------------+------+---------------+----------------

In [69]:
df.printSchema()

root
 |-- Airline: string (nullable = true)
 |-- CRSDepTime: float (nullable = true)
 |-- DepTime: float (nullable = true)
 |-- DepDelay: float (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- ArrTime: float (nullable = true)
 |-- ArrDelay: float (nullable = true)
 |-- ActualElapsedTime: float (nullable = true)
 |-- CRSElapsedTime: float (nullable = true)
 |-- Distance: float (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Flight_Number_Operating_Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- OriginAirportSeqID: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DestAir

In [70]:
df.groupBy("Airline").agg({
    "ArrDelay" : "avg"
}).orderBy(pysql.functions.desc("avg(ArrDelay)")).show(truncate=False)

+-----------------------------------------+---------------------+
|Airline                                  |avg(ArrDelay)        |
+-----------------------------------------+---------------------+
|Commutair Aka Champlain Enterprises, Inc.|16.065892574398116   |
|Allegiant Air                            |15.28626500420386    |
|JetBlue Airways                          |10.610667687240975   |
|Mesa Airlines Inc.                       |8.97437805312025     |
|Air Wisconsin Airlines Corp              |7.345608955800039    |
|GoJet Airlines, LLC d/b/a United Express |7.0096414686825055   |
|Spirit Air Lines                         |6.646201008171151    |
|Frontier Airlines Inc.                   |6.523572190295634    |
|Southwest Airlines Co.                   |5.501016596711463    |
|SkyWest Airlines Inc.                    |4.737164006364127    |
|United Air Lines Inc.                    |3.405323276940503    |
|American Airlines Inc.                   |3.0679115279223104   |
|Envoy Air

In [73]:
df.select("Cancelled").agg(pysql.functions.avg("Cancelled")).show()

+--------------------+
|      avg(Cancelled)|
+--------------------+
|0.017588762507979014|
+--------------------+



In [71]:
df.groupBy("Airline").agg({
    "Cancelled" : "avg",
    "Diverted" : "avg"
}).orderBy(pysql.functions.desc("avg(Cancelled)")).show(truncate=False)

+-----------------------------------------+--------------------+---------------------+
|Airline                                  |avg(Cancelled)      |avg(Diverted)        |
+-----------------------------------------+--------------------+---------------------+
|GoJet Airlines, LLC d/b/a United Express |0.03548210099263422 |0.002228023211346291 |
|Allegiant Air                            |0.03304251775528344 |0.0021573855938419586|
|Commutair Aka Champlain Enterprises, Inc.|0.03246549969999739 |0.003704380038087288 |
|Air Wisconsin Airlines Corp              |0.030016447368421052|0.001644736842105263 |
|Mesa Airlines Inc.                       |0.02980805937830514 |0.0029476371965529544|
|Spirit Air Lines                         |0.029582830357282833|0.0021634502328060578|
|American Airlines Inc.                   |0.02237645624179283 |0.002838135304366247 |
|Envoy Air                                |0.02223256214051949 |0.0026040953896563452|
|Southwest Airlines Co.                   |

In [23]:
spark.stop()